In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN,GRU,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [17]:
# Load the dataset
df = pd.read_csv('..//..//dataset//final_dataset.csv')  # Replace with your dataset path
# df=pd.read_excel('..//..//dataset//Dataset_IoV.xlsx')

# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [18]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(320000, 1, 6) (320000,) (400000, 1, 6)


# Model

In [19]:
def build_deep_rnn_bi_lstm():
    model = Sequential()
    
    # Add multiple SimpleRNN layers for Deep RNN
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))


    model.add(SimpleRNN(units=50,activation='relu', return_sequences=True))
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(units=50, activation='relu',return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(units=50,activation='relu', return_sequences=True)))
    model.add(Dropout(0.1))
   
   

    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])

    return model


In [20]:
# Early stopping callback
early_stopping = EarlyStopping(
    monitor='mse',  
    patience=3,   
    restore_best_weights=True, 
    verbose=1
)
# Model checkpoint callback
best_weights_filepath ='..//models//best_model.keras'

model_checkpoint = ModelCheckpoint(
    best_weights_filepath,  
    monitor='val_mse',  
    mode='min',       
    save_best_only=True,   
    verbose=1
)
# callbacks=[model_checkpoint,early_stopping]

# Instantiate and train the model
model = build_deep_rnn_bi_lstm()
import time
batch_size = X_train.shape[0]  # Use full dataset per batch to match experimental setup
start_time=time.time()
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=128,
    steps_per_epoch=None,
    # steps_per_epoch=3,
    # validation_split=0.2,
    verbose=1
)
end_time = time.time()

Epoch 1/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 31s 9ms/step - loss: 0.0628 - mae: 0.0464 - mse: 0.0207
Epoch 2/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 1.5979e-04 - mae: 9.3658e-05 - mse: 3.7225e-05
Epoch 3/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 3.7239e-05 - mae: 2.0904e-05 - mse: 8.0568e-06
Epoch 4/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 9.0220e-05 - mae: 2.7625e-05 - mse: 1.6604e-05
Epoch 5/50
1305/2500 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 4.4777e-05 - mae: 1.5302e-05 - mse: 8.0208e-06

KeyboardInterrupt: 

In [ ]:
# Calculate total training time
total_time = end_time - start_time  # in seconds

# Number of training samples
num_samples = len(X_train)

# Print results
print(f"Total Training Time: {total_time:.2f} seconds")
aes_ms = aes/1000
print(f"AES: {aes_ms:.2f} ms/sample")

In [ ]:

from sklearn.metrics import mean_squared_error
import numpy as np

# Make predictions
y_pred = model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')

2500/2500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
Mean Squared Error (MSE): 0.0000000001387556
Root Mean Squared Error (RMSE): 0.0000117794556951


In [ ]:
print(y_pred)
y_pred=y_pred.flatten()
print(y_pred)

[[9.821613e-07]
 [9.999979e-01]
 [9.821613e-07]
 ...
 [9.999979e-01]
 [9.730572e-07]
 [9.999979e-01]]
[9.821613e-07 9.999979e-01 9.821613e-07 ... 9.999979e-01 9.730572e-07
 9.999979e-01]


In [ ]:
# accuracy
y_pred = np.round(y_pred)
accuracy = np.mean(y_test == y_pred)
print(f'Accuracy: {accuracy:.16f}')

Accuracy: 1.0000000000000000


In [ ]:
ones_count_actual = np.count_nonzero(y_test)
zeros_count_actual = len(y_test) - ones_count_actual

ones_count_predicted = np.count_nonzero(y_pred)
zeros_count_predicted = len(y_pred) - ones_count_predicted

print(f'Actual: ones={ones_count_actual}, zeros={zeros_count_actual}')
print(f'Predicted: ones={ones_count_predicted}, zeros={zeros_count_predicted}')

Actual: ones=39866, zeros=40134
Predicted: ones=39866, zeros=40134


In [ ]:
# import matplotlib.pyplot as plt

# # Visualizing the loss, MSE, and MAE
# plt.figure(figsize=(15, 5))

# # Loss plot
# plt.subplot(1, 3, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MSE plot
# plt.subplot(1, 3, 2)
# plt.plot(history.history['mse'], label='Train MSE')
# plt.plot(history.history['val_mse'], label='Validation MSE')
# plt.title('Mean Squared Error')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MAE plot
# plt.subplot(1, 3, 3)
# plt.plot(history.history['mae'], label='Train MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Mean Absolute Error')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# plt.tight_layout()
# plt.show()


In [ ]:
#saving the model
model.save('..//..//models//deep_rnn_bi_lstm_lot_of_zeros.keras')  # Replace with your model path